<a href="https://colab.research.google.com/github/Jundo22l/Jundo22l/blob/main/Resnet%2BCIFAR10_0719.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import AdamW
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

# CIFAR-10 데이터셋에 맞춘 데이터 전처리 및 로드
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # w정규화
])

training_data = CIFAR10(root="./", train=True, download=True, transform=transform)
test_data = CIFAR10(root="./", train=False, download=True, transform=transform)

train_loader = DataLoader(training_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# BasicBlock 및 ResNet 모델 정의
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3):
        super(BasicBlock, self).__init__()
        self.c1 = nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.c2 = nn.Conv2d(out_channels, out_channels, kernel_size, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.downsample = None
        if in_channels != out_channels:
            self.downsample = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1)
            self.bn_downsample = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        x_ = x
        x = self.c1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.c2(x)
        x = self.bn2(x)
        if self.downsample is not None:
            x_ = self.downsample(x_)
            x_ = self.bn_downsample(x_)
        x += x_
        x = self.relu(x)
        return x

class ResNet(nn.Module):
    def __init__(self, block, num_classes=10):
        super(ResNet, self).__init__()
        self.b1 = block(in_channels=3, out_channels=64)
        self.b2 = block(in_channels=64, out_channels=128)
        self.b3 = block(in_channels=128, out_channels=128)
        self.b4 = block(in_channels=128, out_channels=256)
        self.b5 = block(in_channels=256, out_channels=256)
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(256 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.b1(x)
        x = self.pool(x)
        x = self.b2(x)
        x = self.b3(x)
        x = self.pool(x)
        x = self.b4(x)
        x = self.b5(x)
        x = self.pool(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# 학습 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
model = ResNet(block=BasicBlock)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=0.01)  # AdamW 사용

# Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# 학습 루프
epochs = 15

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    scheduler.step()

    current_lr = scheduler.get_last_lr()[0]
    print(f"Epoch {epoch+1}/{epochs}, Loss 값: {running_loss/len(train_loader)}, Learning Rate: {current_lr}")

# 모델 평가
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"정확도: {100 * correct / total}%")

100%|██████████| 170498071/170498071 [00:01<00:00, 104809234.14it/s]


Extracting ./cifar-10-python.tar.gz to ./
Files already downloaded and verified
Epoch 1/15, Loss 값: 1.9600951616721385, Learning Rate: 0.01
Epoch 2/15, Loss 값: 1.3884017987324453, Learning Rate: 0.01
Epoch 3/15, Loss 값: 1.0859399872362767, Learning Rate: 0.01
Epoch 4/15, Loss 값: 0.9426564863880577, Learning Rate: 0.01
Epoch 5/15, Loss 값: 0.8216971210049241, Learning Rate: 0.01
Epoch 6/15, Loss 값: 0.7134205414663495, Learning Rate: 0.01
Epoch 7/15, Loss 값: 0.6408171863735789, Learning Rate: 0.01
Epoch 8/15, Loss 값: 0.5875648905706528, Learning Rate: 0.01
Epoch 9/15, Loss 값: 0.5467912609238759, Learning Rate: 0.01
Epoch 10/15, Loss 값: 0.5176796570153492, Learning Rate: 0.001
Epoch 11/15, Loss 값: 0.3609920300238425, Learning Rate: 0.001
Epoch 12/15, Loss 값: 0.31408481913454395, Learning Rate: 0.001
Epoch 13/15, Loss 값: 0.29621379630035144, Learning Rate: 0.001
Epoch 14/15, Loss 값: 0.2766284690240917, Learning Rate: 0.001
Epoch 15/15, Loss 값: 0.25947721713625105, Learning Rate: 0.001
정확도: 

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import AdamW
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

# CIFAR-10 데이터셋에 맞춘 데이터 전처리 및 로드
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # w정규화
])

training_data = CIFAR10(root="./", train=True, download=True, transform=transform)
test_data = CIFAR10(root="./", train=False, download=True, transform=transform)

train_loader = DataLoader(training_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# BasicBlock 및 ResNet 모델 정의
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3):
        super(BasicBlock, self).__init__()
        self.c1 = nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.c2 = nn.Conv2d(out_channels, out_channels, kernel_size, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.downsample = None
        if in_channels != out_channels:
            self.downsample = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1)
            self.bn_downsample = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        x_ = x
        x = self.c1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.c2(x)
        x = self.bn2(x)
        if self.downsample is not None:
            x_ = self.downsample(x_)
            x_ = self.bn_downsample(x_)
        x += x_
        x = self.relu(x)
        return x

class ResNet(nn.Module):
    def __init__(self, block, num_classes=10):
        super(ResNet, self).__init__()
        self.b1 = block(in_channels=3, out_channels=64)
        self.b2 = block(in_channels=64, out_channels=128)
        self.b3 = block(in_channels=128, out_channels=128)
        self.b4 = block(in_channels=128, out_channels=256)
        self.b5 = block(in_channels=256, out_channels=256)
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(256 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.b1(x)
        x = self.pool(x)
        x = self.b2(x)
        x = self.b3(x)
        x = self.pool(x)
        x = self.b4(x)
        x = self.b5(x)
        x = self.pool(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# 학습 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
model = ResNet(block=BasicBlock)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=0.01)  # AdamW 사용

# Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# 학습 루프
epochs = 30

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    scheduler.step()

    current_lr = scheduler.get_last_lr()[0]
    print(f"Epoch {epoch+1}/{epochs}, Loss 값: {running_loss/len(train_loader)}, Learning Rate: {current_lr}")

# 모델 평가
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"정확도: {100 * correct / total}%")

Files already downloaded and verified
Files already downloaded and verified
Epoch 1/30, Loss 값: 2.063007611600334, Learning Rate: 0.01
Epoch 2/30, Loss 값: 1.4512173114225382, Learning Rate: 0.01
Epoch 3/30, Loss 값: 1.2166289658192784, Learning Rate: 0.01
Epoch 4/30, Loss 값: 1.0660103633428168, Learning Rate: 0.01
Epoch 5/30, Loss 값: 0.9685887392524564, Learning Rate: 0.01
Epoch 6/30, Loss 값: 0.8548957917772596, Learning Rate: 0.01
Epoch 7/30, Loss 값: 0.7411945664593022, Learning Rate: 0.01
Epoch 8/30, Loss 값: 0.6802914263037465, Learning Rate: 0.01
Epoch 9/30, Loss 값: 0.62916877736216, Learning Rate: 0.01
Epoch 10/30, Loss 값: 0.5914450063158179, Learning Rate: 0.001
Epoch 11/30, Loss 값: 0.43143274705580736, Learning Rate: 0.001
Epoch 12/30, Loss 값: 0.3921162660621926, Learning Rate: 0.001
Epoch 13/30, Loss 값: 0.36950021767822067, Learning Rate: 0.001
Epoch 14/30, Loss 값: 0.35707017935602864, Learning Rate: 0.001
Epoch 15/30, Loss 값: 0.34339382731929763, Learning Rate: 0.001
Epoch 16/30

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import Adam
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

# CIFAR-10 데이터셋에 맞춘 데이터 전처리 및 로드
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 정규화
])

training_data = CIFAR10(root="./", train=True, download=True, transform=transform)
test_data = CIFAR10(root="./", train=False, download=True, transform=transform)

train_loader = DataLoader(training_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# ResidualBlock 및 ResNet 모델 정의
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.stride = stride
        self.in_channels = in_channels
        self.out_channels = out_channels

        self.conv_block = nn.Sequential(
            nn.Conv2d(self.in_channels, self.out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(self.out_channels),
            nn.ReLU(),

            nn.Conv2d(self.out_channels, self.out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(self.out_channels)
        )

        if self.stride != 1 or self.in_channels != self.out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, self.out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.out_channels)
            )

    def forward(self, x):
        out = self.conv_block(x)
        if self.stride != 1 or self.in_channels != self.out_channels:
            x = self.downsample(x)
        out = F.relu(x + out)
        return out

class ResNet(nn.Module):
    def __init__(self, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.base = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )

        self.layer1 = self.make_layer(64, num_blocks[0], stride=1)
        self.layer2 = self.make_layer(128, num_blocks[1], stride=2)
        self.layer3 = self.make_layer(256, num_blocks[2], stride=2)
        self.layer4 = self.make_layer(512, num_blocks[3], stride=2)

        self.gap = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, out_channels, num_block, stride):
        strides = [stride] + [1] * (num_block - 1)
        layers = []

        for stride in strides:
            block = ResidualBlock(self.in_channels, out_channels, stride)
            layers.append(block)
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.base(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.gap(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

def modeltype(model):
    if model == 'resnet18':
        return ResNet([2, 2, 2, 2])
    elif model == 'resnet34':
        return ResNet([3, 4, 6, 3])

# 학습 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
model = modeltype('resnet18')
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

# 학습 루프
epochs = 10

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    print(f"Epoch {epoch+1}/{epochs}, Loss 값: {running_loss/len(training_data):.4f}")

# 모델 평가
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"정확도: {100 * correct / total:.2f}%")

Files already downloaded and verified
Files already downloaded and verified
